In [58]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split

In [59]:
df = pd.read_csv('dataset/dataset.csv')

In [60]:
df.head()

,filename,label
0,dataset/train/snail/snail (317).jpg,snail
1,dataset/train/wasp/wasp (879).jpg,wasp
2,dataset/train/bees/bees (383).jpg,bees
3,dataset/train/grasshopper/grasshopper (364).jpg,grasshopper
4,dataset/train/weevil/Weevil (161).jpg,weevil


In [61]:
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0  # Normalize pixel values
    return img

In [62]:
# Load dataset
image_paths = df['filename']
labels = df['label']

In [64]:
# Split dataset into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [65]:
# Preprocess images for training set
train_images = [preprocess_image(image_path) for image_path in train_image_paths]

# Preprocess images for testing set
test_images = [preprocess_image(image_path) for image_path in test_image_paths]

# Convert labels to numerical format
label_to_index = {label: i for i, label in enumerate(set(labels))}
train_labels = [label_to_index[label] for label in train_labels]
test_labels = [label_to_index[label] for label in test_labels]

# Create TensorFlow datasets for training and testing
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [66]:
# Shuffle and batch the training dataset
train_dataset = train_dataset.shuffle(buffer_size=len(train_image_paths)).batch(128)

# Batch the testing dataset (no need to shuffle)
test_dataset = test_dataset.batch(128)

In [67]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [68]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(12, activation='softmax')(x)

In [69]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [70]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [71]:
model.fit(train_dataset, epochs=1)

35/35 ━━━━━━━━━━━━━━━━━━━━ 446s 13s/step - accuracy: 0.2251 - loss: 3.6126


In [72]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

# Print evaluation metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 120s 13s/step - accuracy: 0.5334 - loss: 1.5330
Test Loss: 1.5213327407836914
Test Accuracy: 0.5241128206253052
